In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

2024-02-10 21:37:06.622665: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-10 21:37:06.790956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 21:37:06.790987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 21:37:06.816098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 21:37:06.873033: I tensorflow/core/platform/cpu_feature_guar

In [2]:
AUTO = tf.data.experimental.AUTOTUNE

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100%|█████████████████████████████████████▉| 2.70G/2.70G [02:47<00:00, 18.8MB/s]
100%|██████████████████████████████████████| 2.70G/2.70G [02:47<00:00, 17.3MB/s]


In [3]:
from sklearn.model_selection import train_test_split
train_gen = image_dataset_from_directory(directory="Data/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                         image_size=(256, 256))
test_gen = image_dataset_from_directory(directory="Data/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                        image_size=(256, 256))


rescale = Rescaling(scale=1.0/255)
train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

Found 70295 files belonging to 38 classes.


2024-02-10 21:38:12.531557: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-10 21:38:12.638376: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-10 21:38:12.638415: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-10 21:38:12.640490: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-10 21:38:12.640527: I external/local_xla/xla/stream_executor

Found 17572 files belonging to 38 classes.


In [4]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(256, 256, 3)))
model.add(keras.layers.MaxPooling2D(3, 3))

model.add(keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(keras.layers.MaxPooling2D(2, 2))


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(1568, activation="relu"))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(38, activation="softmax"))


opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 85, 85, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 42, 42, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 112896)            0         
                                                                 
 dense (Dense)               (None, 1568)              1

In [5]:
ep = 10
history = model.fit_generator(train_gen,
          validation_data=test_gen,
          epochs = ep)

Epoch 1/10


2024-02-10 21:38:31.813223: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-10 21:38:32.082575: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-10 21:38:33.091646: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fa7a84cd9d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-10 21:38:33.091673: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-02-10 21:38:33.100378: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1707581313.177837     641 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2197/2197 [==============================] - 176s 79ms/step - loss: 1.3954 - accuracy: 0.6006 - val_loss: 0.6235 - val_accuracy: 0.8206
Epoch 2/10
2197/2197 [==============================] - 167s 76ms/step - loss: 0.6548 - accuracy: 0.8008 - val_loss: 0.4058 - val_accuracy: 0.8770
Epoch 3/10
2197/2197 [==============================] - 173s 79ms/step - loss: 0.4599 - accuracy: 0.8565 - val_loss: 0.3007 - val_accuracy: 0.9065
Epoch 4/10
2197/2197 [==============================] - 777s 354ms/step - loss: 0.3477 - accuracy: 0.8912 - val_loss: 0.2452 - val_accuracy: 0.9228
Epoch 5/10
2197/2197 [==============================] - 167s 76ms/step - loss: 0.2757 - accuracy: 0.9125 - val_loss: 0.2176 - val_accuracy: 0.9301
Epoch 6/10
2197/2197 [==============================] - 166s 76ms/step - loss: 0.2199 - accuracy: 0.9301 - val_loss: 0.1991 - val_accuracy: 0.9347
Epoch 7/10
2197/2197 [==============================] - 167s 76ms/step - loss: 0.1791 - accuracy: 0.9421 - val_loss: 0.1797 - va

In [6]:
import pickle
pickle.dump(model,open('model.pkl','wb'))

In [7]:
model.save('model.h5')